# Julemanden har brug for din hjælp!

## Boilerplate

In [ ]:
%load_ext autoreload
%autoreload 2
%pip install -q pytorch_lightning einops
# TODO: more dependencies below. Move to requirements.txt

In [ ]:
from pathlib import Path

import albumentations as A
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import config
import utils
from annoterer import Annoterer
from training import træn_model
from data import GaveDatasæt

## Den er gal i julemandens værksted!

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 4))
axs[0].imshow(Image.open('images_448/217ca0a5-3648-441e-9577-eaee74a27937.jpg'))
axs[0].set_title('ok')
axs[1].imshow(Image.open('images_448/f605b583-3523-469a-a9ef-e3306d4f1232.jpg'))
axs[1].set_title('ikke ok')
for ax in axs:
    ax.axis('off')
plt.tight_layout()

In [ ]:
annoterer = Annoterer(
    debug=True,
)
display(annoterer.widget)

In [ ]:
img_fps = list(config.images_folder.glob('*.jpg'))
img_fps = np.random.choice(img_fps, 20, replace=False)
images = [Image.open(fp) for fp in img_fps]

fig, axs = plt.subplots(2, 5, figsize=(10, 4))
for ax, img in zip(axs.reshape(-1), images):
    ax.imshow(img)
    ax.axis('off')
plt.tight_layout(pad=0.5)

In [ ]:
# Vis eksempler fra det datasæt, vi vil træne vores model på:
augmenteringer = [
    #A.RandomResizedCrop(224, 224, scale=(0.5, 1)),
    #A.ColorJitter(),    
]

dataset = GaveDatasæt(utils.load_annotations(), augs=augmenteringer)
fig, axs = plt.subplots(2, 5, figsize=(12, 5))
idxs = np.random.permutation(len(dataset))
for i, ax in zip(idxs, axs.reshape(-1)):
    img, ok = dataset[i]
    ax.imshow(Image.fromarray(dataset.denormalize(img)))
    ax.axis('off')
    ax.set_title('ok' if ok else 'ikke ok')


In [ ]:
model, version = træn_model(
    epochs=20, lr=1e-4, 
    start_fra_prætrænede_vægte=False,
    frys_backbone=False,
    augmenteringer=[], 
    n_train=100, n_val=100,
)
plot_log(version)